Using kernel `conda_pytorch_latest_p36`

We take inspiration from [this paper](https://arxiv.org/pdf/2104.14690.pdf)

In [1]:
# !pip install cloudpathlib
# !pip install s3fs
# !pip install transformers
# !pip install pytorch-lightning

# Import

In [2]:
import sys
sys.path.append('../../../')

In [3]:
from pathlib import Path
import os
import random
import json
from datetime import datetime

In [4]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
import torch
from torch import nn

import sagemaker
from sagemaker import get_execution_role
import boto3
import torch.nn as nn
from sklearn.preprocessing import MultiLabelBinarizer
from torch.utils.data import DataLoader,Dataset
from tqdm import tqdm

import boto3
import pytorch_lightning as pl
import torch.nn as nn
from transformers import (
    AutoModelForSequenceClassification,
    BertForMaskedLM,
    DistilBertForMaskedLM,
    AutoModel,
    AutoTokenizer,
    AdamW,
    get_linear_schedule_with_warmup,
    Trainer,
    TrainingArguments
)
from torch.utils.data import DataLoader,Dataset

In [5]:
from deep.constants import *
from deep.utils import *

In [6]:
%load_ext autoreload
%autoreload 2

## Data

In [7]:
def preprocessing(df):
    df = df.copy()
    df['sectors'] = df['sectors'].apply(eval)    
    df['pillars'] = df['pillars'].apply(eval)
    df['pillars'] = df['pillars'].apply(lambda x: list(set(x)))
    df['subpillars'] = df['subpillars'].apply(eval)
    return df

In [8]:
column = 'pillars'
classes=PILLARS
text_column = 'excerpt'
text_column_mlm = 'excerpt_mlm'
label_column = 'labels'

In [9]:
train = preprocessing(pd.read_csv(LATEST_DATA_PATH / 'data_v0.4.4_train.csv', index_col=0))
val = preprocessing(pd.read_csv(LATEST_DATA_PATH / 'data_v0.4.4_val.csv', index_col=0))
test = preprocessing(pd.read_csv(LATEST_DATA_PATH / 'data_v0.4.4_test.csv', index_col=0))

In [10]:
classes = PILLARS
len_longest_class =  max([len(x.split()) for x in classes])
append_string = [
    f'[SEP] Is this sentence about {c}? [MASK]'
    for c in classes
]
append_string = ' '.join(append_string)

In [11]:
def preprocess_mlm(df):
    df = df.copy()
    df = df[df[column].apply(len)>0]
    df = df[df['language']=='en']
    df[label_column] = [[1 if y in x else 0 for y in classes] for x in df[column]]
    df[text_column_mlm] = df[text_column] + append_string
    return df

In [12]:
train_df = preprocess_mlm(train)
val_df = preprocess_mlm(val)
test_df = preprocess_mlm(test)

# Model

In [13]:
model_name = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = DistilBertForMaskedLM.from_pretrained(model_name)

In [14]:
class LMBFFDataset(Dataset):
    def __init__(self, tokenizer, df, text_col, label_col, append_string):
        self.tokenizer = tokenizer
        self.df = df
        self.texts = df[text_col].values
        self.labels = self.compute_labels(df[label_col])
        
    def compute_labels(self, labels):
        yes_token = self.tokenizer.convert_tokens_to_ids('yes')
        no_token = self.tokenizer.convert_tokens_to_ids('no')
        labels = [[yes_token if y else no_token for y in label] for label in labels]
        return torch.tensor(labels)

    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        text = self.texts[idx] 
        inputs = self.tokenizer.encode_plus(
            text,
            truncation=True,
            padding='max_length',
            return_tensors='pt',
        )
        inputs['input_ids'] = inputs['input_ids'].squeeze(0)
        labels = self.labels[idx]
        inputs['labels'] = self.mlm_labels(inputs, labels)
        return inputs

    def mlm_labels(self, inputs, label):
        labels = inputs['input_ids'].clone()
        labels[labels != tokenizer.mask_token_id] = -100
        labels[labels == tokenizer.mask_token_id] = label
        return labels

In [21]:
train_dataset = LMBFFDataset(tokenizer, train_df, text_column_mlm, label_column, append_string)
val_dataset = LMBFFDataset(tokenizer, val_df, text_column_mlm, label_column, append_string)
test_dataset = LMBFFDataset(tokenizer, test_df, text_column_mlm, label_column, append_string)

In [16]:
trainer_arguments = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    learning_rate=5e-5,
)

In [17]:
trainer = Trainer(
    model=model, 
    args=trainer_arguments, 
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [22]:
a = trainer.predict(test_dataset)

RuntimeError: CUDA out of memory. Tried to allocate 4.43 GiB (GPU 0; 11.17 GiB total capacity; 5.44 GiB already allocated; 4.40 GiB free; 6.44 GiB reserved in total by PyTorch)

In [ ]:
sacacsvdvvfbdf

In [ ]:
trainer.train()

In [ ]:
optimizer = AdamW(model.parameters(), lr=5e-5)

In [ ]:
for batch in tqdm(train_dataloader):
    outputs = model(**batch)
    loss = outputs.loss
    loss.backward()
    
    optimizer.step()
    optimizer.zero_grad()

In [ ]:
outputs

In [ ]:
input_['input_ids'].shape

In [ ]:
tokenizer.convert_ids_to_tokens(4650)

In [ ]:
tokenizer.convert_tokens_to_ids('humanitarian')

In [ ]:
outputs = model(**input_)

In [ ]:
out = outputs.logits.detach()[0]

In [ ]:
tokenizer.convert_ids_to_tokens(out.argmax(axis=1))

## Sagemaker Prep

### Session

In [ ]:
sess = sagemaker.Session(default_bucket=DEV_BUCKET.name)
role = 'AmazonSageMaker-ExecutionRole-20210519T102514'

### Bucket upload

In [ ]:
sample = False

if sample:
    df = df.sample(100)
    
job_name = f"pytorch-{formatted_time()}-fastai-entail-english"
input_path = DEV_BUCKET / 'training' / 'input_data' / job_name

s3_data = str(input_path / 'df.pickle')


df.to_pickle(s3_data, protocol=4)

### Estimator Definition

In [ ]:
instances = [
    'ml.p2.xlarge',
    'ml.p3.2xlarge'
]

In [ ]:
input_path

In [ ]:
from sagemaker.pytorch import PyTorch

hyperparameters={
    'epochs': 3,
    'train_batch_size': 64,
    'learning_rate': 0.02,
    'text_col': text_column,
    'label_col': label_column,
    'multi_category': 0
}

estimator = PyTorch(
    entry_point='train.py',
    source_dir=str(SCRIPTS_TRAINING_PATH / 'stefano/multiclass-fastai'),
    output_path=str(DEV_BUCKET / 'models/'),
    code_location=str(input_path),
    instance_type='ml.p3.2xlarge',
    instance_count=1,
    role=role,
    framework_version='1.8',
    py_version='py36',
    hyperparameters = hyperparameters,
    job_name=job_name,
#     train_instance_count=2,
#     train_instance_type="ml.c4.xlarge",
)

In [ ]:
fit_arguments = {
    'train': str(input_path),
    'test': str(input_path)
}

In [ ]:
estimator.fit(fit_arguments, job_name=job_name, wait=False)

In [ ]:
from sagemaker import TrainingJobAnalytics

# Captured metrics can be accessed as a Pandas dataframe
df = TrainingJobAnalytics(training_job_name=estimator.latest_training_job.name).dataframe()
df.head(10)

In [ ]:
df